# Mean Reversion Strategy on Currency Pairs

In [ ]:
# Import the required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt  
%matplotlib inline
plt.style.use('seaborn-darkgrid')

In [ ]:
# Fix the backtesting period

end = pd.datetime.now().date()
start = end - pd.Timedelta(days=1*252)

In [ ]:
# Create a class for backtesting pairs
class mean_reversion_pairs:
    
    # Create object attributes/methods
    def __init__(self,x,y,start,end,lookback,std_dev):
        
        self.x = x # Independent variable
        self.y = y # Dependent variable
        self.start =start 
        self.end = end
        self.lookback = lookback # Lookback period to consider moving average
        self.std_dev = std_dev # Number of standard deivations away to create bollinger band
        
        self.fetch_data()
        self.hedge_ratio()
        self.ADFtest()
        self.indicators()
        self.positions()
        self.returns()
        self.calc_drawdown()
        
    # To fetch the data from yahoo finance for both stocks x and y
    def fetch_data(self):
        
        self.dfx = yf.download(self.x,start,end)
        self.dfy = yf.download(self.y,start,end)
        
        
        # Create a single dataframe to concat only the closing prices for backtesting purpose
        self.df = pd.concat([self.dfx['Adj Close'],self.dfy['Adj Close']],axis=1)
        self.df.columns =[self.x,self.y]
        self.df.index = pd.to_datetime(self.df.index)
        
    
    # Calculate hedge ratio by running ordinary least square regression
    def hedge_ratio(self):
        import statsmodels.api as sm
        self.model = sm.OLS(self.df[self.y].iloc[:120],self.df[self.x].iloc[:120])
        self.model = self.model.fit()
        self.df['spread'] = self.df[self.y] - self.model.params[0] * self.df[self.x]
        return self.model.params[0]
    
    # Perform ADF test on the spread of the pair and return the T statistic value
    def ADFtest(self):
        from statsmodels.tsa.stattools import adfuller
        self.ADF = adfuller(self.df['spread'].dropna(),maxlag=1)
        return self.ADF[0]
   
    # Compute the required indicators for bollinger band
    def indicators(self):
        
        # Moving Average
        self.df['moving_average'] = self.df['spread'].rolling(self.lookback,center=False).mean()
        
        # Moving Standard Deviation
        self.df['moving_std_dev'] = self.df['spread'].rolling(self.lookback,center=False).std()

        # Upper band and lower band
        self.df['upper_band'] = self.df['moving_average'] + self.std_dev*self.df['moving_std_dev']
        self.df['lower_band'] = self.df['moving_average'] - self.std_dev*self.df['moving_std_dev']
    
    # Compute the positions based on the signal
    def positions(self):
        
        # Long positions
        self.df['long_entry'] = self.df['spread'] < self.df['lower_band']
        self.df['long_exit'] = self.df['spread'] >= self.df['moving_average']

        self.df['positions_long'] = np.nan
        self.df.loc[self.df['long_entry'], 'positions_long'] = 1
        self.df.loc[self.df['long_exit'], 'positions_long'] = 0
        self.df['positions_long'] = self.df['positions_long'].fillna(method='ffill')

        # Short positions
        self.df['short_entry'] = self.df['spread'] > self.df['upper_band']
        self.df['short_exit'] = self.df['spread'] <= self.df['moving_average']

        self.df['positions_short'] = np.nan
        self.df.loc[self.df['short_entry'], 'positions_short'] = -1
        self.df.loc[self.df['short_exit'], 'positions_short'] = 0
        self.df['positions_short'] = self.df['positions_short'].fillna(method='ffill')
        
        # Positions
        self.df['positions'] = self.df['positions_long'] + self.df['positions_short']
    
    # Calculate the strategy returns
    def returns(self):
        
        self.df['percentage_change'] = (self.df['spread'] - self.df['spread'].shift(1))/(self.model.params[0]*self.df[self.x] + self.df[self.y])
        self.df['strategy_returns'] = self.df['positions'].shift(1) * self.df['percentage_change']
        self.df['cumulative_returns'] = (self.df['strategy_returns']+1).cumprod()
        print("The total strategy returns are " ,((self.df['cumulative_returns'].iloc[-1]-1)*100))
        return (self.df['cumulative_returns'].iloc[-1]-1)*100
    
    # Calculate the max drawdown of the returns
    def calc_drawdown(self):
        # Calculate the running maximum
        self.running_max = np.maximum.accumulate(self.df['cumulative_returns'].dropna())
        # Ensure the value never drops below 1
        self.running_max[self.running_max < 1] = 1
        # Calculate the percentage drawdown
        self.drawdown = (self.df['cumulative_returns'])/self.running_max - 1
        return self.drawdown.min()*100

In [ ]:
# Run the strategy by considering different cement stocks of same cap with different look back period and number of standard deviations away from the mean
Currency_X_list = ['EURUSD=X','AUDUSD=X','EURCAD=X','EURCHF=X','CADUSD=X','CHFUSD=X','HKD=X','KRW=X']
Currency_Y_list = ['EURUSD=X','AUDUSD=X','EURCAD=X','EURCHF=X','CADUSD=X','CHFUSD=X','HKD=X','KRW=X']

X=[]
Y=[]

lookback_list =range(5,25,5)
stdev_list = [1,1.5,2,2.5]

lookbackperiod =[]
stdev_range = []
Hedge_ratio = []
ADF_Tstat = []
Strat_Returns = []
Drawdown = []
Correlation = []

for i in Currency_X_list:
    for j in Currency_Y_list:
        for k in lookback_list:
            for l in stdev_list:
                if i == j:
                    continue
                else:
                
            
                    Pairs = mean_reversion_pairs(i,j,start,end,k,l)
                    X.append(i)
                    Y.append(j)
                    lookbackperiod.append(k)
                    stdev_range.append(l)
                    Hedge_ratio.append(Pairs.hedge_ratio())
                    ADF_Tstat.append(Pairs.ADFtest())
                    Strat_Returns.append(Pairs.returns())
                    Drawdown.append(Pairs.calc_drawdown())

In [ ]:
# Convert the results in to dataframe for validation
Pair_Strategy = pd.DataFrame({'Stock_X':X,'Stock_Y':Y,'Lookback' :lookbackperiod,'Std dev':stdev_range,'Hedge_Ratio':Hedge_ratio,'ADF Tstat': ADF_Tstat,'Correlation':Correlation,'Strategy Returns':Strat_Returns,'Max Drawdown': Drawdown})


In [ ]:
# Sort the dataframe based on highest returns
Pair_Strategy.sort_values(by='Strategy Returns',ascending=False)


In [ ]:
Pair_Strategy.to_csv("Test.csv")